# Part I

In [1]:
import pyaudio
import numpy as np
CHUNK = 1024
resolution = pyaudio.paInt16 
CHANNELS = 1 
sampling_rate = 16000
record_time=5
Expected_num_of_chunks= int(sampling_rate/CHUNK*record_time) 
p=pyaudio.PyAudio()
stream = p.open(format=resolution, channels=CHANNELS, rate=sampling_rate, input=True, frames_per_buffer=CHUNK)
frames=[]
for i in range(0,Expected_num_of_chunks): 
    data = stream.read(CHUNK)
    frames.append(data)      
array_frames=np.array(frames)
intframes=np.frombuffer(array_frames, np.int16)
stream.stop_stream()
stream.close()
p.terminate()
import IPython.display as ipd
ipd.Audio(intframes,rate=16000)

# Part II

In [11]:
import keyboard
import time
import wave
import pyaudio
import numpy as np
import IPython.display as ipd
CHUNK = 1600
resolution = pyaudio.paInt16 
CHANNELS = 1 
sampling_rate = 16000

def hit_to_talk():
    keyboard.wait('s')
    time.sleep(1)
    print("Please talk")
def EnergyPerSampleInDecibel(audioframe):
    audioframe=np.frombuffer(np.array(audioframe),np.int16)
    np_audioframe=np.array(audioframe,dtype='int64')
    energy= 10*np.log10(np.sum(np.square(np_audioframe)))
    return energy
def find_ini_background(audioframe):
    background_energy=0 #我把循环移到下面stream开始后的那个background位置了
    background_energy+=EnergyPerSampleInDecibel(audioframe)
    return background_energy
def classifyFrame(audioframe,background,level, First_run):
    forgetfactor=1
    isSpeech=False
    current = EnergyPerSampleInDecibel(audioframe)
    if First_run:
        First_run=False
        level=EnergyPerSampleInDecibel(audioframe)
    else:
        level=((level*forgetfactor)+current)/(1+forgetfactor)
    threshold=4
    adjustment=0.05
    print('Background',background,'current',current, 'level', level)
    if current < background:
        background = current
    else:
        background+=(current - background)*adjustment
    print('level-background', level-background)
    if (level < background): level = background
    if (level - background > threshold): isSpeech = True
    return level,isSpeech
def record():
    CHUNK = 1600
    resolution = pyaudio.paInt16
    CHANNELS = 1
    sampling_rate = 16000
    stop_record_time=1.5
    stop_speak_time=0
    stop_speak_per_loop=0.1
    isSpeech=False
    p=pyaudio.PyAudio()
    stream = p.open(format=resolution, channels=CHANNELS, rate=sampling_rate, input=True, frames_per_buffer=CHUNK)
    frames=[]
    intframes=[]
    level=0
    First_run=True
    background=0
    while True:
        audioframe=stream.read(CHUNK)
        if First_run:
            for i in range(0,10):#新的计算background的位置
                background+=find_ini_background(audioframe)
            background=background/10
        frames.append(audioframe)
        level,isSpeech=classifyFrame(audioframe,background,level,First_run)
        First_run=False
        if isSpeech==True:
            stop_speak_time=0
        else:
            stop_speak_time+=0.1
        if stop_speak_time>=stop_record_time:
            wf = wave.open('1.wav', 'wb')
            wf.setnchannels(CHANNELS)
            wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
            wf.setframerate(sampling_rate)
            wf.writeframes(b''.join(frames))
            wf.close()
            print("Audio data saved as 1.wav")
            stream.stop_stream()
            stream.close()
            p.terminate()
            break
    return intframes
def start():
    hit_to_talk()
    record()

In [12]:
a=start()

Please talk
Background 55.46536494478609 current 55.46536494478608 level 55.46536494478608
level-background 0.0
Background 55.46536494478609 current 54.01584904459341 level 54.74060699468974
level-background 0.7247579500963326
Background 55.46536494478609 current 49.435884874603005 level 52.08824593464637
level-background 2.6523610600433685
Background 55.46536494478609 current 52.668781870049486 level 52.37851390234793
level-background -0.2902679677015527
Background 55.46536494478609 current 54.657441174462576 level 53.66311152225603
level-background -0.9943296522065452
Background 55.46536494478609 current 51.04555391240513 level 52.85149754343385
level-background 1.8059436310287182
Background 55.46536494478609 current 51.55718534045289 level 52.20434144194337
level-background 0.6471561014904808
Background 55.46536494478609 current 51.30719636562952 level 51.755768903786446
level-background 0.4485725381569239
Background 55.46536494478609 current 51.11977904962384 level 51.4377739767051